# Review Instructions

Please review the MSv4 processing_set class https://github.com/casangi/xradio/blob/main/src/xradio/vis/_processing_set.py

The processing set is a loose collection of MSv4 which might come from multiple MSv2 (or ASDMS). Consequently, arbitrary ids are avoided in favor of descriptive strings.

Run the notebook using:
- ```partition_scheme=['FIELD_ID']``` 
and 
- ```partition_scheme=[]``` 

## Key Questions to Answer
1) Is there additional information to display in the summary table?
2) Are the docstrings sufficient?
3) Are there missing data selection use cases?
4) ...

# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 213-fix-ps-selection
cd xradio
pip install -e .
```

On macOS it is required to pre-install python-casacore using ```bash conda install -c conda-forge python-casacore```.

# Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

[2024-08-13 11:11:14,186]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-13 11:11:14,186]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

[2024-08-13 11:11:15,240]     INFO  graphviper:  File exists: ALMA_uid___A002_X1003af4_X75a3.split.avg.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-13 11:11:15,323]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-13 11:11:15,323]     INFO  graphviper:  Module path: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11//site-packages/ 
[2024-08-13 11:11:15,326]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63329 instead
  warnings.warn(


[2024-08-13 11:11:16,105]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:63330' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63329/status,
Dashboard: http://127.0.0.1:63329/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63330,Workers: 4
Dashboard: http://127.0.0.1:63329/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:63341,Total threads: 1
Dashboard: http://127.0.0.1:63343/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:63333,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
import os

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"
os.system("rm -rf "+out_file)

partition_scheme=['FIELD_ID'] #can be ephemeris_interpolate=True/False, Default
#partition_scheme=[] #must be ephemeris_interpolate=True, Rapid OTF mode

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
    ephemeris_interpolate=True,
    partition_scheme=partition_scheme
)

[2024-08-13 11:11:16,407]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-13 11:11:16,985]     INFO      client:  Number of partitions: 96 
[2024-08-13 11:11:16,985]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [7] 
[2024-08-13 11:11:17,257]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [16], FIELD [0], SCAN [7] 
[2024-08-13 11:11:17,549]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [17], FIELD [0], SCAN [7] 
[2024-08-13 11:11:17,719]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [19], FIELD [2], SCAN [6] 
[2024-08-13 11:11:17,863]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [20 99], FIELD [0], SCAN [8] 
[2024-08-13 11:11:18,041]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [21 48 75], FIELD [3], SCAN [8] 
[2024-08-13 11:11:18,265]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [22 49 76], FIELD [4],

# Inspect Processing Set

In [ ]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

# Using ps.sel() with summary table column names

In [ ]:
#Note that no selection is applied on the MS data so even if field_name=['Sun_10_10','Sun_10_11'] all the fields are kept.
ps.sel(field_coords='Ephemeris',field_name=['Sun_10_10','Sun_10_11']).summary() #Select all Ephemeris data and where any of the fields are 'Sun_10_10' or 'Sun_10_11'.



# Using ps.sel() with query and summary table column names

In [ ]:
ps.sel(query="start_frequency > 2.46e11",field_coords='Ephemeris',field_name=['Sun_10_10','Sun_10_11']).summary() #Select all Ephemeris data and where any of the fields are 'Sun_10_10' or 'Sun_10_11'.

# Ordering MSv4 Selection

In [ ]:
summary_df = ps.sel(obs_mode='OBSERVE_TARGET#ON_SOURCE').summary()
summary_df=summary_df.sort_values(by=['start_frequency'],ascending=True)

summary_df

In [ ]:
#First MS is then give by:
first_ms_name = summary_df['name'][0]
ps[first_ms_name]

# Sel by number

In [ ]:

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

min_freq = min(ps.summary()['start_frequency'])
ps.sel(start_frequency=min_freq).summary()

# Require exact match in selection criteria

In [ ]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(name='ALMA_uid___A002_X1003af4_X75a3.split.avg_01',string_exact_match=True).summary()

# Allow for partial match in string. (select all MSv4 with field_names with "Sun_10" in the name).

In [ ]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(field_name='Sun_10',string_exact_match=False).summary()

# Allow for partial match in string. (select all MSv4 with ALMA_uid___A002_X1003af4_X75a3.split.avg in the name).

This example does nothing but would be useful for a PS created from MSv4s that are created from different MSv2s.

In [ ]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(name='ALMA_uid___A002_X1003af4_X75a3.split.avg',string_exact_match=False).summary()